In [37]:
import numpy
from flask import Flask
from flask import request
from keras.layers import Dense
from keras.models import Sequential
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from tensorflow.python.keras.backend import set_session
from keras.models import load_model
from keras.wrappers.scikit_learn import KerasClassifier
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import tensorflow

app = Flask(__name__)

global graph
graph = tensorflow.get_default_graph()
sess = tensorflow.Session()

def larger_model():
        model = Sequential()
        model.add(Convolution2D(32,(3,3),input_shape=(64,64,3),activation = 'relu'))
        model.add(MaxPooling2D((2,2)))
        model.add(Flatten())
        model.add(Dense(units=128,activation='relu'))
        model.add(Dense(units=1, activation='sigmoid'))                
        model.compile(optimizer = 'adam',loss='binary_crossentropy',metrics=['accuracy'])
        return model

#load model
classifire = larger_model()  
set_session(sess)
classifire.load_weights('saved_model.h5')  

@app.route("/predict",methods =['POST'])
def predictions(): 
    #preProcess the image 
    image = tensorflow.keras.preprocessing.image.load_img('data_set/test/PNEUMONIA/person1_virus_7.jpeg',target_size=(64,64))
    input_arr = tensorflow.keras.preprocessing.image.img_to_array(image)
    input_arr = numpy.array([input_arr])  # Convert single image to a batch.
    with graph.as_default():
                 set_session(sess)                 
                 prediction = classifire.predict(input_arr)[0]    
                 responseValue= "Server Error..."
    if prediction == 1:
                responseValue = "Pneumonia"
    elif prediction == 0:
                responseValue = "Normal"
        
    return str("prediction"+": "+responseValue)

if __name__ == '__main__':
    app.run(debug=False, port=5000)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
INFO:werkzeug: * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [13/Nov/2020 15:52:52] "POST /predict HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [13/Nov/2020 15:52:52] "POST /predict HTTP/1.1" 200 -
